In [253]:
import numpy as np
from copy import deepcopy

Project focused on calculating homographies, using QR decomp to calculate the K, R, C matrices. Also computes the reporjection error for the given homography

In [254]:
image_pts = np.array(
    [
        [757,  213],
        [758,  415],
        [758,  686],
        [759,  966],
        [1190, 172],
        [329, 1041],
        [1204, 850],
        [340,  159]
    ]
)


world_pts = np.array(
    [
        [0, 0,  0],
        [0, 3,  0],
        [0, 7,  0],
        [0, 11, 0],
        [7, 1,  0],
        [0, 11, 7],
        [7, 9,  0],
        [0, 1,  7]
    ]
)


In [255]:
def find_p(image_points, world_points):
    
    a_mat = np.array([])
    
    rows = []
    
    for i in range(len(image_points)):
        xc = image_points[i][0]
        yc = image_points[i][1]
        
        Xw = world_points[i][0]
        Yw = world_points[i][1]
        Zw = world_points[i][2]

        rows.append([Xw, Yw, Zw, 1, 0,  0,  0,  0, -xc*Xw, -xc*Yw, -xc*Zw, -xc])
        rows.append([0,  0,  0,  0, Xw, Yw, Zw, 1, -yc*Xw, -yc*Yw, -yc*Zw, -yc])
        
    
    a_mat = np.array(rows)
    aTa = a_mat.T@a_mat
    eigvals, eigvecs = np.linalg.eig(aTa)
    
    p = eigvecs[:, np.argmin(eigvals)]
    p = p.reshape(3, 4)
    return p

In [256]:
def gram_schmidt(p):
    
    d = min(p.shape)
    M = p[:d, :d]
  
    R = np.array([])
    
    for i in range(d):
        ai = M[i, :]
        
        # Ui = ai - sum(ai dot Uk) for k = 0 to i-1 
        if len(R) != 0:
            projections = np.sum(ai@R)
            ri = (ai - projections).reshape(-1, 1)
            R = np.concatenate((R, ri), axis = 1)
            
        else:
            R = ai.reshape(-1, 1)
            
    for col in range(len(R)):
        R[col] /= np.linalg.norm(R[col])
    
    
    K = M@(np.linalg.inv(R))
    
    return K, R
    
        

In [257]:
p = find_p(image_pts, world_pts)
k, r, = gram_schmidt(p)
c = -1*p[:, -1].reshape(-1, 1)

In [258]:
print(f"P")
print(p)
print(f"K")
print(k)
print(f"R")
print(r)
print(f"C")
print(c)


P
[[ 3.62233659e-02 -2.21521076e-03 -8.83242916e-02  9.54088881e-01]
 [-2.53833189e-02  8.30555704e-02 -2.80016309e-02  2.68827013e-01]
 [-3.49222323e-05 -3.27184804e-06 -3.95667606e-05  1.26053750e-03]]
K
[[6.65015252e+01 5.67915347e+01 5.47131292e+01]
 [2.10639296e+01 1.80835431e+01 1.73661530e+01]
 [2.97709235e-02 2.54386242e-02 2.45470542e-02]]
R
[[ 8.04395548e-01 -5.94093471e-01 -8.65795843e-04]
 [-2.71087094e-02  9.99632487e-01 -8.97983431e-05]
 [-9.48908662e-01 -3.15550520e-01 -4.68767958e-04]]
C
[[-0.95408888]
 [-0.26882701]
 [-0.00126054]]


In [259]:
def get_reproj_err(image_points, world_pts, p):
    
    ones = np.ones(shape=(len(world_pts), 1))
    homogeneous_world_pts = np.concatenate((world_pts, ones), axis=1)
    reproj_pts = np.zeros_like(image_points).astype(np.float32)
        
    for i in range(len(homogeneous_world_pts)):
        w_pt = homogeneous_world_pts[i]
    
        r_pt = p@w_pt
        r_pt = r_pt/r_pt[-1]
        reproj_pts[i] = r_pt[:2]

    reproj_err = np.sqrt(np.sum((reproj_pts-image_pts)**2, axis = 1))

    return reproj_err



In [260]:
reproj_err = get_reproj_err(image_pts, world_pts, p)
np.average(reproj_err)

0.4703265841741417